In [3]:
import yfinance as yf  
aapl= yf.Ticker("aapl")

yfinance.Ticker object <AAPL>

In [7]:
aapl_historical = aapl.history(start="2023-09-05", end="2023-09-11", interval="1d")
aapl_historical.head

<bound method NDFrame.head of                                  Open        High         Low       Close  \
Date                                                                        
2023-09-05 00:00:00-04:00  188.279999  189.979996  187.610001  189.699997   
2023-09-06 00:00:00-04:00  188.399994  188.850006  181.470001  182.910004   
2023-09-07 00:00:00-04:00  175.179993  178.210007  173.539993  177.559998   
2023-09-08 00:00:00-04:00  178.350006  180.240005  177.789993  178.179993   

                              Volume  Dividends  Stock Splits  
Date                                                           
2023-09-05 00:00:00-04:00   45280000        0.0           0.0  
2023-09-06 00:00:00-04:00   81755800        0.0           0.0  
2023-09-07 00:00:00-04:00  112488800        0.0           0.0  
2023-09-08 00:00:00-04:00   65551300        0.0           0.0  >